<a href="https://colab.research.google.com/github/bekkopapa/study/blob/main/lottery_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install 'h5py==2.10.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached h5py-2.10.0.tar.gz (301 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for h5py: filename=h5py-2.10.0-cp310-cp310-linux_x86_64.whl size=5619499 sha256=5f4cf640ca29af5b5dcf7bd7ef5643a3f2eaae3e68aaf4692e317bb2d57793cd
  Stored in directory: /root/.cache/pip/wheels/21/bc/58/0d0c6056e1339f40188d136cd838c6554d9c17545196dd9110
Successfully built h5py
  Attempting uninstall: h5py
    Found existing installation: h5py 3.8.0
    Uninstalling h5py-3.8.0:
      Successfully uninstalled h5py-3.8.0


In [58]:
import numpy as np

rows = np.loadtxt("./lottery_data.csv", delimiter=",")
row_count = len(rows)
print(row_count)

157


In [59]:
# 원본 데이터 로드
import pandas as pd

data = pd.read_csv('lottery_data.csv')
rows = data.to_numpy()

# 당첨 번호 정렬
sorted_numbers = np.array([np.sort(row[1:7]) for row in rows])


In [60]:
def numbers2sequence(numbers):
    return np.array(numbers, dtype=int)

def sequence2numbers(sequence):
    return list(sequence)

# 당첨 번호를 정수 시퀀스로 변환
sequences = list(map(numbers2sequence, numbers))

x_samples = sequences[0:row_count]
y_samples = sequences[1:row_count]

print("sequences")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

print("numbers")
print("X[0]: " + str(sequence2numbers(x_samples[0])))
print("Y[0]: " + str(sequence2numbers(y_samples[0])))


sequences
X[0]: [0 7 1 7 1 2]
Y[0]: [7 3 9 5 8 6]
numbers
X[0]: [0, 7, 1, 7, 1, 2]
Y[0]: [7, 3, 9, 5, 8, 6]


In [ ]:
import tensorflow as tf
from tensorflow import keras

# 데이터 준비
# x_samples와 y_samples는 이전 단계에서 이미 준비한 것으로 가정합니다.
x_train = np.array(x_samples[:-1])
y_train = np.array(y_samples)

# 모델 생성
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10, output_dim=32, input_length=6), # 임베딩 레이어 추가
    keras.layers.LSTM(128, return_sequences=False),
    keras.layers.Dense(6, activation='relu') # 출력 레이어의 크기를 6으로 변경
])

# 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# 모델 훈련
history = model.fit(x_train, y_train, epochs=500, batch_size=1)


In [68]:
loss, accuracy = model.evaluate(x_train, y_train)
print('Model loss:', loss)
print('Model accuracy:', accuracy)


5/5 [==============================] - 0s 4ms/step - loss: 0.0019 - accuracy: 0.8774
Model loss: 0.001933670835569501
Model accuracy: 0.8774193525314331


In [76]:
predicted_sequence = model.predict(x_test)
print("Predicted output sequence: " + str(predicted_sequence))

predicted_sequence = np.ceil(predicted_sequence)
print("Rounded predicted output sequence: " + str(predicted_sequence))


1/1 [==============================] - 0s 19ms/step
Predicted output sequence: [[2.9710042 2.4851174 4.218071  5.1372676 5.6773643 2.9734757]]
Rounded predicted output sequence: [[3. 3. 5. 6. 6. 3.]]


In [75]:
model.save("lottery_model.h5")


In [84]:
x_new = np.array([[2, 3, 4, 5, 7, 8]])
y_pred = model.predict(x_new)
print("Predicted output sequence:", np.round(y_pred))


1/1 [==============================] - 0s 22ms/step
Predicted output sequence: [[6. 7. 7. 0. 6. 6.]]
